# CNN Übung
Für diese Übung werden Sie mit dem folgenden Datensatz arbeiten: <a href='https://www.kaggle.com/zalando-research/fashionmnist'>Fashion-MNIST</a>, welches hier abgerufen werden kann: <a href='https://pytorch.org/docs/stable/torchvision/index.html'><tt><strong>torchvision</strong></tt></a>. Ähnlich dem MNIST Datensatz, haben wir einen training set von 60,000 examples und einen test set von 10,000 examples. Jedes example ist ein 28x28 grayscale Bild, welches 1 label von 10 Klassen besitzt:

0. T-shirt/top
1. Trouser
2. Pullover
3. Dress
4. Coat
5. Sandal
6. Shirt
7. Sneaker
8. Bag
9. Ankle boot


In [18]:
# Folgendes müssen Sie zuerst ausführen, um alle Voraussetzungen zu haben!!

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

# transform = transforms.ToTensor() #hier hatte ich nur 10% 

# Die Transformationsfunktion `Normalize(mean, std)` normalisiert die Pixelwerte des Bildes.
# Hier werden die Mittelwerte (0.5,) und die Standardabweichungen (0.5,) angegeben.
# Dadurch werden die Pixelwerte zwischen -1 und 1 normalisiert.
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# Trainingsdaten laden
train_data = datasets.FashionMNIST(root='../Data', train=True, download=True, transform=transform)

# Testdaten laden
test_data = datasets.FashionMNIST(root='../Data', train=False, download=True, transform=transform)

# Liste der Klassennamen
class_names = ['T-shirt','Hose','Pullover','Kleid','Mantel','Sandalen','Shirt','Turnschuhe','Tasche','Stiefel']


In [28]:
train_data

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: ../Data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5,), std=(0.5,))
           )

Aufgabe 1: Erstellen Sie die Data Loaders

Sowohl train_loader als auch test_loader sollen Sie erstellen mit Batch Sizes = 10

In [19]:
# Code hier:

learning_rate = 0.001
batch_size = 10
epochs = 5

# Trainingsdaten-Loader erstellen
train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)

# Testdaten-Loader erstellen
test_loader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False)


Aufgabe 2: CNN Definieren

Definineren Sie eine CNN mit 2 conv layers, 2 pooling layers und 2 fully connected layers. Sie können beliebig viele Neuronen nutzen. 

In [21]:
# CNN-Modell definieren
class ConvolutionalNetwork(nn.Module):
    def __init__(self):
        super().__init__()

        # Erster Convolutional Layer: Eingangskanäle: 1 (schwarz-weißes Bild), Ausgangskanäle: 16, Kernelgröße: 3x3, Stride: 1, Padding: 1
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()  # ReLU-Aktivierungsfunktion nach dem ersten Convolutional Layer
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)  # Max-Pooling Layer mit Kernelgröße 2x2 und Stride 2

        # Zweiter Convolutional Layer: Eingangskanäle: 16, Ausgangskanäle: 32, Kernelgröße: 3x3, Stride: 1, Padding: 1
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()  # ReLU-Aktivierungsfunktion nach dem zweiten Convolutional Layer
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)  #Zweiter Max-Pooling Layer mit Kernelgröße 2x2 und Stride 2

        # Erster Fully Connected Layer: Eingangsgröße: 7x7x32 (aus der vorherigen Convolutional Layer), Ausgangsgröße: 128
        self.fc1 = nn.Linear(7*7*32, 128)
        self.relu3 = nn.ReLU()  # ReLU-Aktivierungsfunktion nach dem ersten Fully Connected Layer

        # Zweiter Fully Connected Layer: Eingangsgröße: 128, Ausgangsgröße: 10 (Anzahl der Klassen)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        # Vorwärtsdurchlauf des Modells
        x = self.pool1(self.relu1(self.conv1(x)))  # Erste Convolutional Layer, ReLU und Pooling
        x = self.pool2(self.relu2(self.conv2(x)))  # Zweite Convolutional Layer, ReLU und Pooling
        x = x.view(x.size(0), -1)  # Umformen der Daten für den Fully Connected Layer
        x = self.relu3(self.fc1(x))  # Erster Fully Connected Layer mit ReLU
        x = self.fc2(x)  # Zweiter Fully Connected Layer

        return F.log_softmax(x, dim=1)

torch.manual_seed(101)
# Modellinitialisierung
model = ConvolutionalNetwork()


ConvolutionalNetwork(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu1): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1568, out_features=128, bias=True)
  (relu3): ReLU()
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

Aufgabe 3: Parameter Anzahl

Finden Sie heraus wie viele Parameter das CNN Modell beseitzt

In [22]:
# Code hier:

# Berechnung der Gesamtanzahl der Parameter im Modell
total_params = sum(p.numel() for p in model.parameters())

# Ausgabe der Anzahl der Parameter
print(f'Anzahl der Parameter: {total_params}')


Anzahl der Parameter: 206922


Aufgabe 4: Loss Function & Optimizer Definieren

Definieren Sie Loss Function & Optimizer. Wir haben Cross Entropy Loss und Adam (lr=0.001) genutzt, Sie können aber
auch andere nutzen wenn Sie möchten. 

In [24]:
# Code hier:

import torch.optim as optim

# Loss-Funktion (Kreuzentropie) und Optimizer (Adam) definieren
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


Aufgabe 5: Modell Trainieren

Vervollständigen Sie folgenden Code um das Modell zu trainieren:

In [27]:
# epochs = 5

# Trainingsschleife über die Anzahl der Epochen
for i in range(epochs):
    # Iteration über die Trainingsdaten im train_loader
    for X_train, y_train in train_loader:
        
        # Vorwärtsdurchlauf (Berechnung der Modellvorhersagen und des Loss)
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        
        # Rückwärtsdurchlauf und Optimierung (Anpassung der Modellparameter)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # OPTIONAL print statement
    print(f'{i+1} von {epochs} Epochen abgeschlossen')


1 of 5 epochs completed
2 of 5 epochs completed
3 of 5 epochs completed
4 of 5 epochs completed
5 of 5 epochs completed


Aufgabe 6: Modell Evaluieren

Vervollständigen Sie folgenden Code um das Modell zu evaluieren:

In [30]:
# Modell in Evaluationsmodus versetzen
model.eval()

# Deaktivieren der Berechnung des Gradienten
with torch.no_grad():
    
    correct = 0
    total = 0

    # Iteration über die Testdaten im test_loader
    for X_test, y_test in test_loader:
        # Vorwärtsdurchlauf (Berechnung der Modellvorhersagen)
        outputs = model(X_test)
        _, predicted = torch.max(outputs.data, 1)
        total += y_test.size(0)
        correct += (predicted == y_test).sum().item()

# Berechnung der Testgenauigkeit
accuracy = 100 * correct / total
print(f'Testgenauigkeit: {accuracy:.2f}%')
# Alternative Ausgabe:
# print(f'Testgenauigkeit: {correct}/{len(test_data)} = {correct*100/len(test_data):.3f}%')


Testgenauigkeit: 91.30%
